## Modelling

**using this there is no need to reload your packages every time when you will make any improvement to them**

In [1]:
%load_ext autoreload

In [2]:
autoreload 2

**import necessary libraries**

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import sys
import warnings

**set warnings options**

In [4]:
warnings.filterwarnings('ignore')

#### to have ease in importing modules

In [5]:
sys.path.insert(0, '/Users/mjasiecz/PycharmProjects/new_offer_success_predictor/src')

**set pandas options**

In [6]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 200

**balancing dataset**

In [68]:
# from imblearn.over_sampling import SMOTE
# from imblearn.combine import SMOTEENN

**import packages for manage and preprocess data**

In [9]:
from data.data_manager import DataManager
from data.data_processor import DataProcessor, TestDataProcessor
from ml_preprocessing.categorical_encoders import LeaveOneOutEncoder
from ml_preprocessing.encoding_helpers import LOOGridSearchCV

**load data pipeline**

In [10]:
DM = DataManager()
train_df, test_df = DM.load_train_test()

**ml data preprocessing**

In [11]:
DP = DataProcessor(train_df=train_df)
processed_train_df = DP.perform_initial_features_engineering()
TDP = TestDataProcessor(not_processed_train_df=train_df,
                        processed_train_df=processed_train_df,
                        test_df=test_df,
                        sneaky_peaky=True)
processed_test_df = TDP.perform_initial_features_engineering()

Imputing row 1/1031 with 1 missing, elapsed time: 0.223
Imputing row 1/1289 with 0 missing, elapsed time: 0.316
Imputing row 1033/1289 with 0 missing, elapsed time: 0.319


In [12]:
processed_train_df.head()

,cc_len,cc_startswith,log_salary,log_age_knn,log_estimated_expenses_knn,log_offer_value_knn,scaled_salary,scaled_age_knn,scaled_estimated_expenses_knn,scaled_offer_value_knn,nan_age,not_nan_age,target,offer_class,gender,age,phone_calls,emails,salary,customer_type,center,age_knn,estimated_expenses_knn,offer_value_knn
name,,,,,,,,,,,,,,,,,,,,,,,,
8550AB469CB2445,ELSE,ELSE,7.329474,3.401259,8.882808,5.716940,-0.345530,-0.297159,1.362250,3.035567,True,False,False,Premium,female,NaN,1,2,1524.58,C,B,30.001848,7207.0,303.973257
07355EE27DD1493,ELSE,ELSE,7.170120,3.465736,8.209308,4.160055,-0.389281,-0.111655,-0.492006,-1.109564,False,True,False,High,male,32.0,0,0,1300.00,S,A,32.000000,3675.0,64.075055
034E73A251554F0,ELSE,ELSE,7.346010,3.177471,8.465900,4.687503,-0.340578,-0.855654,0.072355,-0.340546,True,False,True,Premium,female,NaN,1,1,1550.00,Q,B,23.986022,4750.0,108.581750
0AF961B4AC7A439,ELSE,ELSE,6.583299,3.323884,7.244942,5.046620,-0.501704,-0.504546,-1.685828,0.470071,True,False,False,Premium,male,NaN,0,1,722.92,C,A,27.767979,1401.0,155.495957
8535BBCA690A4AE,ELSE,ELSE,8.639336,3.405772,7.652546,4.798852,0.458072,-0.284561,-1.315712,-0.119564,True,False,True,Premium,male,NaN,0,1,5649.58,S,B,30.137541,2106.0,121.371033


In [13]:
processed_test_df.head()

,cc_len,cc_startswith,log_salary,log_age_knn,log_estimated_expenses_knn,log_offer_value_knn,scaled_salary,scaled_age_knn,scaled_estimated_expenses_knn,scaled_offer_value_knn,nan_age,not_nan_age,target,age,center,customer_type,emails,gender,offer_class,phone_calls,salary,age_knn,estimated_expenses_knn,offer_value_knn
name,,,,,,,,,,,,,,,,,,,,,,,,
DA3BAEB8BF604EB,ELSE,ELSE,6.594755,3.489576,8.940891,4.963600,-0.501839,-0.035049,1.603528,0.246935,True,False,False,NaN,A,S,1,male,Premium,0,731.25,32.772049,7638.0,143.108044
AEF3DE08DFED4E0,ELSE,ELSE,6.656084,3.433987,8.662851,4.153202,-0.492946,-0.197055,0.631794,-1.130746,False,True,False,31.0,A,S,0,male,Premium,0,777.50,31.000000,5784.0,63.637416
B621BB29484E46D,ELSE,ELSE,6.671501,3.218876,8.434898,5.348081,-0.490623,-0.745594,0.013846,1.410101,False,True,False,25.0,B,S,1,male,Premium,0,789.58,25.000000,4605.0,210.204552
2D0945802F92423,ELSE,ELSE,6.514713,2.890372,8.402680,4.876044,-0.512654,-1.385557,-0.062677,0.038956,False,True,False,18.0,B,Q,1,female,Premium,0,675.00,18.000000,4459.0,131.110924
640ABFC7E49B403,58,ELSE,8.029296,4.025352,8.388450,5.193064,-0.052226,2.088525,-0.095697,0.886816,False,True,False,56.0,B,C,1,male,Medium,0,3069.58,56.000000,4396.0,180.019199


**verify if there is no mistake during preprocessing**

In [14]:
assert not set(processed_train_df.index).intersection(set(processed_test_df.index))

**left to process: categoricalencoding and age binning approach**

**sklearn modelling**

In [274]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

**sklearn models which have predict_proba method**

In [23]:
from sklearn.utils.testing import all_estimators

estimators = all_estimators()

for name, class_ in estimators:
    if hasattr(class_, 'predict_proba'):
        print(name)

AdaBoostClassifier
BaggingClassifier
BayesianGaussianMixture
BernoulliNB
CalibratedClassifierCV
ClassifierChain
ComplementNB
DecisionTreeClassifier
DummyClassifier
ExtraTreeClassifier
ExtraTreesClassifier
GaussianMixture
GaussianNB
GaussianProcessClassifier
GradientBoostingClassifier
GridSearchCV
HistGradientBoostingClassifier
KNeighborsClassifier
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LogisticRegression
LogisticRegressionCV
MLPClassifier
MultiOutputClassifier
MultinomialNB
NuSVC
OneVsRestClassifier
Pipeline
QuadraticDiscriminantAnalysis
RFE
RFECV
RandomForestClassifier
RandomizedSearchCV
SGDClassifier
SVC
VotingClassifier
_BinaryGaussianProcessClassifierLaplace
_ConstantPredictor


In [28]:
columns_to_encode = ['offer_class', 'gender', 'customer_type', 'center', 'phone_calls', 'cc_len', 'cc_startswith']

In [16]:
enc = LeaveOneOutEncoder(train_df=processed_train_df,
                         test_df=processed_test_df,
                         columns_to_encode=columns_to_encode,
                         target_column='target',
                         random_state=42,
                         mean=1,
                         std=0.05)

In [24]:
_, test_df_encoded = enc.fit()

In [25]:
X_test = test_df_encoded.reset_index().drop(columns=['target'])
y_test = test_df_encoded.reset_index()[['target']]

In [33]:
print(processed_train_df.columns.values)

['cc_len' 'cc_startswith' 'log_salary' 'log_age_knn'
 'log_estimated_expenses_knn' 'log_offer_value_knn' 'scaled_salary'
 'scaled_age_knn' 'scaled_estimated_expenses_knn' 'scaled_offer_value_knn'
 'nan_age' 'not_nan_age' 'target' 'offer_class' 'gender' 'age'
 'phone_calls' 'emails' 'salary' 'customer_type' 'center' 'age_knn'
 'estimated_expenses_knn' 'offer_value_knn']


In [35]:
all_train_features = processed_train_df.columns.values

#### estimators

In [69]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [70]:
knn_clf = KNeighborsClassifier()
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
log_clf = LogisticRegression()

In [106]:
knn_param_grid = {'n_neighbors': list(np.linspace(2, 20, 10, endpoint=True).astype(int)),
                  'weights': ['uniform', 'distance'],
                  'metric': ['minkowski'],
                  'p': list(np.linspace(1, 4, 4, endpoint=True).astype(int))}

In [131]:
dt_param_grid = {'max_depth': list(np.linspace(2, 20, 10, endpoint=True).astype(int)),
                 'min_samples_split': list(np.linspace(0.1, 1, 10, endpoint=True)),
                 'max_features': list(np.linspace(2, 8, 4, endpoint=True).astype(int)),
                 'min_samples_leaf': list(np.linspace(0.1, 0.5, 5, endpoint=True)),
                 'random_state': [42]}

In [108]:
rf_param_grid = {'n_estimators': [10, 20, 30, 50, 80, 100, 200, 300],
                 'max_depth': list(np.linspace(2, 8,4, endpoint=True).astype(int)),
                 'max_features': list(np.linspace(2, 8, 4, endpoint=True).astype(int)),
                 'random_state': [42]}

In [109]:
log_param_grid = {'C': [0.01, 0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 1, 2, 5],
                  'random_state': [42]}

**first set of chosen features**

In [110]:
columns_to_drop_from_training1 = ['salary', 'scaled_salary','scaled_age_knn', 'scaled_estimated_expenses_knn',
                                  'scaled_offer_value_knn', 'estimated_expenses_knn', 'offer_value_knn',
                                  'age_knn', 'log_age_knn', 'age']

In [111]:
set(all_train_features).difference(set(columns_to_drop_from_training1))

{'cc_len',
 'cc_startswith',
 'center',
 'customer_type',
 'emails',
 'gender',
 'log_estimated_expenses_knn',
 'log_offer_value_knn',
 'log_salary',
 'nan_age',
 'not_nan_age',
 'offer_class',
 'phone_calls',
 'target'}

In [112]:
LOOGS1_knn = LOOGridSearchCV(train_df=processed_train_df, model=knn_clf, params_grid=knn_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training1)

In [113]:
knn1_acc, knn1_pam, knn1_rec, knn1_cms = LOOGS1_knn.grid_search()

In [123]:
len(knn1_acc)

80

In [116]:
print(max(knn1_acc))

0.7856526429341963


In [117]:
print(max(knn1_rec))

0.654986346986347


In [132]:
LOOGS1_dt = LOOGridSearchCV(train_df=processed_train_df, model=dt_clf, params_grid=dt_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training1, 
                        Xs_train=LOOGS1_knn.Xs_train,
                        ys_train=LOOGS1_knn.ys_train,
                        Xs_val=LOOGS1_knn.Xs_val,
                        ys_val=LOOGS1_knn.ys_val,
                        encoded_df=LOOGS1_knn.encoded_df_)

In [133]:
dt1_acc, dt1_pam, dt1_rec, dt1_cms = LOOGS1_dt.grid_search()

In [134]:
len(dt1_acc)

2000

In [140]:
print(max(dt1_acc))

0.780812344636743


In [143]:
print(dt1_acc[dt1_rec.index(max(dt1_rec))])

0.7332676703719339


In [201]:
print(max(dt1_rec))

0.7827106227106226


In [148]:
dt1_rec.index(max(dt1_rec))

16

In [149]:
dt1_cms[16]

array([[90. , 38. ],
       [17. , 61.2]])

In [151]:
LOOGS1_rf = LOOGridSearchCV(train_df=processed_train_df, model=rf_clf, params_grid=rf_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training1, 
                        Xs_train=LOOGS1_knn.Xs_train,
                        ys_train=LOOGS1_knn.ys_train,
                        Xs_val=LOOGS1_knn.Xs_val,
                        ys_val=LOOGS1_knn.ys_val,
                        encoded_df=LOOGS1_knn.encoded_df_)

In [152]:
rf1_acc, rf1_pam, rf1_rec, rf1_cms = LOOGS1_rf.grid_search()

In [153]:
print(max(rf1_acc))

0.8050654284508232


In [154]:
print(max(rf1_rec))

0.6727938727938728


In [252]:
LOOGS1_log = LOOGridSearchCV(train_df=processed_train_df, model=log_clf, params_grid=log_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training1, 
                        Xs_train=LOOGS1_knn.Xs_train,
                        ys_train=LOOGS1_knn.ys_train,
                        Xs_val=LOOGS1_knn.Xs_val,
                        ys_val=LOOGS1_knn.ys_val,
                        encoded_df=LOOGS1_knn.encoded_df_)

In [253]:
log1_acc, log1_pam, log1_rec, log1_cms = LOOGS1_log.grid_search()

In [254]:
print(max(log1_acc))

0.773054734768538


In [255]:
print(max(log1_rec))

0.6812600732600733


**second set of chosen features**

In [43]:
columns_to_drop_from_training2 = ['salary', 'age_knn', 'log_age_knn', 'age', 'estimated_expenses_knn', 
                                  'offer_value_knn', 'log_estimated_expenses_knn', 'log_offer_value_knn',
                                  'log_salary', 'scaled_age_knn']

In [42]:
set(all_train_features).difference(set(columns_to_drop_from_training2))

{'cc_len',
 'cc_startswith',
 'center',
 'customer_type',
 'emails',
 'gender',
 'nan_age',
 'not_nan_age',
 'offer_class',
 'phone_calls',
 'scaled_age_knn',
 'scaled_estimated_expenses_knn',
 'scaled_offer_value_knn',
 'scaled_salary',
 'target'}

In [160]:
LOOGS2_knn = LOOGridSearchCV(train_df=processed_train_df, model=knn_clf, params_grid=knn_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training2)

In [161]:
knn2_acc, knn2_pam, knn2_rec, knn2_cms = LOOGS2_knn.grid_search()

In [162]:
print(max(knn2_acc))

0.7526851461000892


In [163]:
print(max(knn2_rec))

0.5992707292707292


In [164]:
LOOGS2_dt = LOOGridSearchCV(train_df=processed_train_df, model=dt_clf, params_grid=dt_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training2, 
                        Xs_train=LOOGS2_knn.Xs_train,
                        ys_train=LOOGS2_knn.ys_train,
                        Xs_val=LOOGS2_knn.Xs_val,
                        ys_val=LOOGS2_knn.ys_val,
                        encoded_df=LOOGS2_knn.encoded_df_)

In [165]:
dt2_acc, dt2_pam, dt2_rec, dt2_cms = LOOGS2_dt.grid_search()

In [166]:
print(max(dt2_acc))

0.780812344636743


In [176]:
print(max(dt2_rec))

0.7827106227106226


In [168]:
LOOGS2_rf = LOOGridSearchCV(train_df=processed_train_df, model=rf_clf, params_grid=rf_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training2, 
                        Xs_train=LOOGS2_knn.Xs_train,
                        ys_train=LOOGS2_knn.ys_train,
                        Xs_val=LOOGS2_knn.Xs_val,
                        ys_val=LOOGS2_knn.ys_val,
                        encoded_df=LOOGS2_knn.encoded_df_)

In [169]:
rf2_acc, rf2_pam, rf2_rec, rf2_cms = LOOGS2_rf.grid_search()

In [170]:
print(max(rf2_acc))

0.8060363022372309


In [171]:
print(max(rf2_rec))

0.6727938727938728


In [180]:
LOOGS2_log = LOOGridSearchCV(train_df=processed_train_df, model=log_clf, params_grid=log_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training2, 
                        Xs_train=LOOGS2_knn.Xs_train,
                        ys_train=LOOGS2_knn.ys_train,
                        Xs_val=LOOGS2_knn.Xs_val,
                        ys_val=LOOGS2_knn.ys_val,
                        encoded_df=LOOGS2_knn.encoded_df_)

In [184]:
len(log2_acc)

10

In [181]:
log2_acc, log2_pam, log2_rec, log2_cms = LOOGS2_log.grid_search()

In [182]:
print(max(log2_acc))

0.7759579757047043


In [183]:
print(max(log2_rec))

0.6727825507825508


**third set of chosen features**

In [47]:
original_numerical = ['salary', 'estimated_expenses_knn', 'offer_value_knn', 'age_knn']

In [48]:
processed_train_df[original_numerical].var()

salary                    2.637465e+07
estimated_expenses_knn    3.631811e+06
offer_value_knn           3.352732e+03
age_knn                   1.161376e+02
dtype: float64

In [49]:
columns_to_drop_from_training3 = ['salary', 'age_knn', 'log_age_knn', 'age', 'estimated_expenses_knn', 
                                  'offer_value_knn', 'scaled_estimated_expenses_knn', 'log_offer_value_knn',
                                  'scaled_salary', 'scaled_age_knn']

In [50]:
set(all_train_features).difference(set(columns_to_drop_from_training3))

{'cc_len',
 'cc_startswith',
 'center',
 'customer_type',
 'emails',
 'gender',
 'log_estimated_expenses_knn',
 'log_salary',
 'nan_age',
 'not_nan_age',
 'offer_class',
 'phone_calls',
 'scaled_offer_value_knn',
 'target'}

In [185]:
LOOGS3_knn = LOOGridSearchCV(train_df=processed_train_df, model=knn_clf, params_grid=knn_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training3)

In [186]:
knn3_acc, knn3_pam, knn3_rec, knn3_cms = LOOGS3_knn.grid_search()

In [187]:
print(max(knn3_acc))

0.7710848459265514


In [188]:
print(max(knn3_rec))

0.6172860472860473


In [189]:
LOOGS3_dt = LOOGridSearchCV(train_df=processed_train_df, model=dt_clf, params_grid=dt_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training3, 
                        Xs_train=LOOGS3_knn.Xs_train,
                        ys_train=LOOGS3_knn.ys_train,
                        Xs_val=LOOGS3_knn.Xs_val,
                        ys_val=LOOGS3_knn.ys_val,
                        encoded_df=LOOGS3_knn.encoded_df_)

In [190]:
dt3_acc, dt3_pam, dt3_rec, dt3_cms = LOOGS3_dt.grid_search()

In [191]:
print(max(dt3_acc))

0.780812344636743


In [192]:
print(max(dt3_rec))

0.7827106227106226


In [193]:
LOOGS3_rf = LOOGridSearchCV(train_df=processed_train_df, model=rf_clf, params_grid=rf_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training3, 
                        Xs_train=LOOGS3_knn.Xs_train,
                        ys_train=LOOGS3_knn.ys_train,
                        Xs_val=LOOGS3_knn.Xs_val,
                        ys_val=LOOGS3_knn.ys_val,
                        encoded_df=LOOGS3_knn.encoded_df_)

In [194]:
rf3_acc, rf3_pam, rf3_rec, rf3_cms = LOOGS3_rf.grid_search()

In [195]:
print(max(rf3_acc))

0.8050654284508232


In [196]:
print(max(rf3_rec))

0.6727938727938728


In [197]:
LOOGS3_log = LOOGridSearchCV(train_df=processed_train_df, model=log_clf, params_grid=log_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training3, 
                        Xs_train=LOOGS3_knn.Xs_train,
                        ys_train=LOOGS3_knn.ys_train,
                        Xs_val=LOOGS3_knn.Xs_val,
                        ys_val=LOOGS3_knn.ys_val,
                        encoded_df=LOOGS3_knn.encoded_df_)

In [198]:
log3_acc, log3_pam, log3_rec, log3_cms = LOOGS3_log.grid_search()

In [199]:
print(max(log3_acc))

0.7749871019182965


In [200]:
print(max(log3_rec))

0.6805081585081585


**forth set of chosen features**

In [51]:
all_train_features

array(['cc_len', 'cc_startswith', 'log_salary', 'log_age_knn',
       'log_estimated_expenses_knn', 'log_offer_value_knn',
       'scaled_salary', 'scaled_age_knn', 'scaled_estimated_expenses_knn',
       'scaled_offer_value_knn', 'nan_age', 'not_nan_age', 'target',
       'offer_class', 'gender', 'age', 'phone_calls', 'emails', 'salary',
       'customer_type', 'center', 'age_knn', 'estimated_expenses_knn',
       'offer_value_knn'], dtype=object)

In [203]:
columns_to_encode.remove('center')

In [204]:
columns_to_drop_from_training4 = ['salary', 'age', 'center', 'age_knn', 'estimated_expenses_knn', 'offer_value_knn', 
                                  'log_age_knn', 'log_estimated_expenses_knn', 'log_offer_value_knn', 'log_salary',
                                  'nan_age', 'not_nan_age']

In [205]:
set(all_train_features).difference(set(columns_to_drop_from_training4))

{'cc_len',
 'cc_startswith',
 'customer_type',
 'emails',
 'gender',
 'offer_class',
 'phone_calls',
 'scaled_age_knn',
 'scaled_estimated_expenses_knn',
 'scaled_offer_value_knn',
 'scaled_salary',
 'target'}

In [206]:
LOOGS4_knn = LOOGridSearchCV(train_df=processed_train_df, model=knn_clf, params_grid=knn_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training4)

In [207]:
knn4_acc, knn4_pam, knn4_rec, knn4_cms = LOOGS4_knn.grid_search()

In [208]:
print(max(knn4_acc))

0.7333051920641622


In [209]:
print(max(knn4_rec))

0.592041958041958


In [210]:
LOOGS4_dt = LOOGridSearchCV(train_df=processed_train_df, model=dt_clf, params_grid=dt_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training4, 
                        Xs_train=LOOGS4_knn.Xs_train,
                        ys_train=LOOGS4_knn.ys_train,
                        Xs_val=LOOGS4_knn.Xs_val,
                        ys_val=LOOGS4_knn.ys_val,
                        encoded_df=LOOGS4_knn.encoded_df_)

In [211]:
dt4_acc, dt4_pam, dt4_rec, dt4_cms = LOOGS4_dt.grid_search()

In [212]:
print(max(dt4_acc))

0.7788705970639275


In [213]:
print(max(dt4_rec))

0.7444841824841826


In [214]:
LOOGS4_rf = LOOGridSearchCV(train_df=processed_train_df, model=rf_clf, params_grid=rf_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training4, 
                        Xs_train=LOOGS4_knn.Xs_train,
                        ys_train=LOOGS4_knn.ys_train,
                        Xs_val=LOOGS4_knn.Xs_val,
                        ys_val=LOOGS4_knn.ys_val,
                        encoded_df=LOOGS4_knn.encoded_df_)

In [215]:
rf4_acc, rf4_pam, rf4_rec, rf4_cms = LOOGS4_rf.grid_search()

In [216]:
print(max(rf4_acc))

0.8002204399418413


In [217]:
print(max(rf4_rec))

0.6783769563769563


In [218]:
LOOGS4_log = LOOGridSearchCV(train_df=processed_train_df, model=log_clf, params_grid=log_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training4, 
                        Xs_train=LOOGS4_knn.Xs_train,
                        ys_train=LOOGS4_knn.ys_train,
                        Xs_val=LOOGS4_knn.Xs_val,
                        ys_val=LOOGS4_knn.ys_val,
                        encoded_df=LOOGS4_knn.encoded_df_)

In [219]:
log4_acc, log4_pam, log4_rec, log4_cms = LOOGS4_log.grid_search()

In [220]:
print(max(log4_acc))

0.7759579757047043


In [221]:
print(max(log4_rec))

0.6753466533466532


**fifth set of chosen features**

In [222]:
columns_to_drop_from_training5 = ['salary', 'age', 'center', 'age_knn', 'estimated_expenses_knn', 'offer_value_knn', 
                                  'log_age_knn', 'log_estimated_expenses_knn', 'log_offer_value_knn', 'log_salary',
                                  'nan_age', 'not_nan_age', 'cc_len', 'cc_startswith']

In [223]:
columns_to_encode.remove('cc_len')

In [224]:
columns_to_encode.remove('cc_startswith')

In [225]:
set(all_train_features).difference(set(columns_to_drop_from_training5))

{'customer_type',
 'emails',
 'gender',
 'offer_class',
 'phone_calls',
 'scaled_age_knn',
 'scaled_estimated_expenses_knn',
 'scaled_offer_value_knn',
 'scaled_salary',
 'target'}

In [226]:
LOOGS5_knn = LOOGridSearchCV(train_df=processed_train_df, model=knn_clf, params_grid=knn_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training5)

In [227]:
knn5_acc, knn5_pam, knn5_rec, knn5_cms = LOOGS5_knn.grid_search()

In [228]:
print(max(knn5_acc))

0.7352281787908634


In [229]:
print(max(knn5_rec))

0.6049290709290709


In [230]:
LOOGS5_dt = LOOGridSearchCV(train_df=processed_train_df, model=dt_clf, params_grid=dt_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training5, 
                        Xs_train=LOOGS5_knn.Xs_train,
                        ys_train=LOOGS5_knn.ys_train,
                        Xs_val=LOOGS5_knn.Xs_val,
                        ys_val=LOOGS5_knn.ys_val,
                        encoded_df=LOOGS5_knn.encoded_df_)

In [231]:
dt5_acc, dt5_pam, dt5_rec, dt5_cms = LOOGS5_dt.grid_search()

In [232]:
print(max(dt5_acc))

0.7798414708503353


In [233]:
print(max(dt5_rec))

0.8105181485181484


In [242]:
dt5_rec.index(max(dt5_rec))

90

In [243]:
dt5_cms[90]

array([[72. , 56. ],
       [14.8, 63.4]])

In [244]:
dt5_acc[90]

0.656662445476291

In [234]:
LOOGS5_rf = LOOGridSearchCV(train_df=processed_train_df, model=rf_clf, params_grid=rf_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training5, 
                        Xs_train=LOOGS5_knn.Xs_train,
                        ys_train=LOOGS5_knn.ys_train,
                        Xs_val=LOOGS5_knn.Xs_val,
                        ys_val=LOOGS5_knn.ys_val,
                        encoded_df=LOOGS5_knn.encoded_df_)

In [235]:
rf5_acc, rf5_pam, rf5_rec, rf5_cms = LOOGS5_rf.grid_search()

In [236]:
print(max(rf5_acc))

0.8021574973031284


In [237]:
print(max(rf5_rec))

0.6727938727938728


In [238]:
LOOGS5_log = LOOGridSearchCV(train_df=processed_train_df, model=log_clf, params_grid=log_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training5, 
                        Xs_train=LOOGS5_knn.Xs_train,
                        ys_train=LOOGS5_knn.ys_train,
                        Xs_val=LOOGS5_knn.Xs_val,
                        ys_val=LOOGS5_knn.ys_val,
                        encoded_df=LOOGS5_knn.encoded_df_)

In [239]:
log5_acc, log5_pam, log5_rec, log5_cms = LOOGS5_log.grid_search()

In [240]:
print(max(log5_acc))

0.774991792129825


In [241]:
print(max(log5_rec))

0.6779440559440559


**sixth set of chosen features**

In [64]:
columns_to_drop_from_training6 = ['salary', 'age', 'center', 'age_knn', 'estimated_expenses_knn', 'offer_value_knn', 
                                  'log_age_knn', 'log_estimated_expenses_knn', 'log_offer_value_knn', 'log_salary',
                                  'nan_age', 'not_nan_age', 'cc_len', 'cc_startswith']

In [65]:
set(all_train_features).difference(set(columns_to_drop_from_training6))

{'customer_type',
 'emails',
 'gender',
 'offer_class',
 'phone_calls',
 'scaled_age_knn',
 'scaled_estimated_expenses_knn',
 'scaled_offer_value_knn',
 'scaled_salary',
 'target'}

In [249]:
columns_to_encode.remove('phone_calls')

convert back phone_calls and emails to numerical

In [245]:
num_processed_train_df = processed_train_df.copy(deep=True)

In [251]:
num_processed_train_df['phone_calls'] = num_processed_train_df['phone_calls'].astype(int)
num_processed_train_df['emails'] = num_processed_train_df['emails'].astype(int)

In [256]:
LOOGS6_knn = LOOGridSearchCV(train_df=num_processed_train_df, model=knn_clf, params_grid=knn_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training6,
                        ohe_emails=False)

In [257]:
knn6_acc, knn6_pam, knn6_rec, knn6_cms = LOOGS6_knn.grid_search()

In [258]:
print(max(knn6_acc))

0.7255194409267858


In [259]:
print(max(knn6_rec))

0.5090223110223111


In [260]:
LOOGS6_dt = LOOGridSearchCV(train_df=num_processed_train_df, model=dt_clf, params_grid=dt_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training6, 
                        Xs_train=LOOGS6_knn.Xs_train,
                        ys_train=LOOGS6_knn.ys_train,
                        Xs_val=LOOGS6_knn.Xs_val,
                        ys_val=LOOGS6_knn.ys_val,
                        encoded_df=LOOGS6_knn.encoded_df_,
                        ohe_emails=False)

In [261]:
dt6_acc, dt6_pam, dt6_rec, dt6_cms = LOOGS6_dt.grid_search()

In [262]:
len(dt6_acc)

2000

In [263]:
print(max(dt6_acc))

0.7788705970639275


In [264]:
print(max(dt6_rec))

0.6727938727938728


In [265]:
LOOGS6_rf = LOOGridSearchCV(train_df=num_processed_train_df, model=rf_clf, params_grid=rf_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training6, 
                        Xs_train=LOOGS6_knn.Xs_train,
                        ys_train=LOOGS6_knn.ys_train,
                        Xs_val=LOOGS6_knn.Xs_val,
                        ys_val=LOOGS6_knn.ys_val,
                        encoded_df=LOOGS6_knn.encoded_df_,
                        ohe_emails=False)

In [266]:
rf6_acc, rf6_pam, rf6_rec, rf6_cms = LOOGS6_rf.grid_search()

In [267]:
print(max(rf6_acc))

0.799244875943905


In [268]:
print(max(rf6_rec))

0.6727938727938728


In [269]:
LOOGS6_log = LOOGridSearchCV(train_df=num_processed_train_df, model=log_clf, params_grid=log_param_grid, 
                        columns_to_encode=columns_to_encode,
                        columns_to_drop_from_training=columns_to_drop_from_training6, 
                        Xs_train=LOOGS6_knn.Xs_train,
                        ys_train=LOOGS6_knn.ys_train,
                        Xs_val=LOOGS6_knn.Xs_val,
                        ys_val=LOOGS6_knn.ys_val,
                        encoded_df=LOOGS6_knn.encoded_df_,
                        ohe_emails=False)

In [270]:
log6_acc, log6_pam, log6_rec, log6_cms = LOOGS6_log.grid_search()

In [271]:
print(max(log6_acc))

0.7866422775667182


In [272]:
print(max(log6_rec))

0.6648518148518148


**seventh set of chosen features**

In [387]:
y_train = processed_train_df[['target']]

In [422]:
y_test = test_df[['target']]

In [389]:
columns_to_use_for_training7 = ['customer_type', 'gender', 'emails', 'phone_calls', 'cc_startswith', 'cc_len',
                                'scaled_salary', 'offer_class']#, 'not_nan_age'] #drop not_nan_age to see results, also add roc_auc to metrics?

In [390]:
features_set_seventh = processed_train_df[columns_to_use_for_training7]

In [391]:
test7 = processed_test_df[columns_to_use_for_training7]

In [392]:
features_set_seventh['phone_calls'] = features_set_seventh['phone_calls'].astype(int)
features_set_seventh['emails'] = features_set_seventh['emails'].astype(int)

In [393]:
test7['phone_calls'] = test7['phone_calls'].astype(int)
test7['emails'] = test7['emails'].astype(int)

In [394]:
test7 = pd.get_dummies(test7)

In [395]:
test7.shape

(258, 17)

In [396]:
features_set_seventh = pd.get_dummies(features_set_seventh)

In [397]:
features_set_seventh.shape

(1031, 17)

convert back phone_calls and emails to numerical, one hot encoding on categoricals columns

In [398]:
from sklearn.metrics import recall_score, make_scorer

In [399]:
recall_scorer = make_scorer(recall_score)

In [400]:
knn7 = GridSearchCV(estimator=knn_clf, param_grid= knn_param_grid, cv=5, scoring=recall_scorer)

In [401]:
knn7.fit(features_set_seventh, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'metric': ['minkowski'],
                         'n_neighbors': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
                         'p': [1, 2, 3, 4],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(recall_score), verbose=0)

In [402]:
knn7.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=14, p=2,
                     weights='distance')

In [403]:
knn7.best_params_

{'metric': 'minkowski', 'n_neighbors': 14, 'p': 2, 'weights': 'distance'}

In [404]:
max(knn7.cv_results_['mean_test_score'])

0.6546654489784547

In [405]:
dt7 = GridSearchCV(estimator=dt_clf, param_grid= dt_param_grid, cv=5, scoring=recall_scorer)

In [406]:
dt7.fit(features_set_seventh, y_train)

/Users/mjasiecz/anaconda3/envs/new_offer_predictor/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n...
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
                         'max_features': [2, 4, 6, 8],
             

In [426]:
from sklearn.metrics import classification_report, confusion_matrix

In [408]:
max(dt7.cv_results_['mean_test_score'])

0.7572912544612628

In [418]:
#dt7_predictions = dt7.best_estimator_.predict(test7)

In [462]:
pred = dt7.best_estimator_.predict_proba(test7)[:,1]>0.52789

In [466]:
#dt7.best_estimator_.predict_proba(test7)

In [463]:
print(confusion_matrix(y_test, pred))

[[101  59]
 [ 29  69]]


In [421]:
y_test.shape

(1031, 1)

In [425]:
print(classification_report(y_test, dt7_predictions))

              precision    recall  f1-score   support

       False       0.83      0.49      0.62       160
        True       0.50      0.84      0.63        98

    accuracy                           0.62       258
   macro avg       0.67      0.67      0.62       258
weighted avg       0.71      0.62      0.62       258



In [428]:
print(confusion_matrix(y_test, dt7_predictions))

[[79 81]
 [16 82]]


In [409]:
rf7 = GridSearchCV(estimator=rf_clf, param_grid= rf_param_grid, cv=5, scoring=recall_scorer)

In [410]:
rf7.fit(features_set_seventh, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [411]:
max(rf7.cv_results_['mean_test_score'])

0.6828432201242182

In [412]:
log7 = GridSearchCV(estimator=log_clf, param_grid= log_param_grid, cv=5, scoring=recall_scorer)

In [413]:
log7.fit(features_set_seventh, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.05, 0.1, 0.2, 0.5, 0.6, 0.7, 1, 2, 5],
                         'random_state': [42]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(recall_score), verbose=0)

In [414]:
max(log7.cv_results_['mean_test_score'])

0.690487771016691

**eighth set of chosen features**

convert back phone_calls and emails to numerical, one hot encoding on categoricals columns

In [290]:
columns_to_use_for_training8 = ['customer_type',  'emails', 'phone_calls', 'scaled_salary', 'gender', 'scaled_offer_value_knn']

In [291]:
features_set_eighth = num_processed_train_df[columns_to_use_for_training8]

In [294]:
features_set_eighth.head()

,customer_type,emails,phone_calls,scaled_salary,gender,scaled_offer_value_knn
name,,,,,,
8550AB469CB2445,C,2,1,-0.345530,female,3.035567
07355EE27DD1493,S,0,0,-0.389281,male,-1.109564
034E73A251554F0,Q,1,1,-0.340578,female,-0.340546
0AF961B4AC7A439,C,1,0,-0.501704,male,0.470071
8535BBCA690A4AE,S,1,0,0.458072,male,-0.119564


In [299]:
features_set_eighth = pd.get_dummies(features_set_eighth)

In [ ]:
GridSearchCV(estimator= , param_grid= )